## Tables for "A natural hazard risk modelling approach to human displacement - frontiers & challenges"

In this notebook we create the tables and human displacement risk values featured in the manuscript titled "A natural hazard risk modelling approach to human displacement - frontiers & challenges" by Meiler et al., 2025.

In [1]:
# load libraries
import pandas as pd
import numpy as np
import geopandas as gpd

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# paths (to be adjusted when running on different machines)
results_path = '/Users/simonameiler/Documents/work/03_code/repos/global-displacement-risk/data/results/'

### Supplementary Table 2. Global average annual human displacement values.

In [3]:
def calculate_future_changes_multiple_scenarios(input_files, baseline_year, baseline_scenario, future_years, future_scenarios, metric):
    """
    Calculate and display baseline totals, future absolute values, and percentage changes with future scenarios/years as columns.
    Includes a final row with totals across all hazards, where percentage changes are correctly calculated from total values.

    Parameters:
        input_files (list of str): List of file paths to the input CSV files.
        baseline_year (int): The baseline year (e.g., 2020).
        baseline_scenario (str): The baseline scenario (e.g., 'current').
        future_years (list of int): List of future years to consider (e.g., [2050, 2100]).
        future_scenarios (list of str): List of future scenarios to consider (e.g., ['optimistic', 'pessimistic']).
        metric (str): The metric column to calculate changes (e.g., 'AAD').

    Returns:
        pd.DataFrame: A summary table with future scenarios/years as columns, showing baseline totals, absolute values, and percentage changes.
    """
    results = []  # Store results for all hazards
    totals = {}  # Store aggregated totals for the last row

    # Initialize totals for aggregation
    totals['Hazard'] = 'Total Across All Hazards'
    totals['Baseline Total'] = 0  # To accumulate baseline totals
    for future_year in future_years:
        for future_scenario in future_scenarios:
            abs_column_name = f"{future_scenario}_{future_year}_Value"
            perc_column_name = f"{future_scenario}_{future_year}_Change (%)"
            totals[abs_column_name] = 0  # Initialize for sum
            totals[perc_column_name] = 0  # Placeholder for percentage change

    for file_path in input_files:
        try:
            # Extract hazard name from the file name
            hazard = file_path.split('/')[-1].split('_')[0]
            
            # Load the CSV file
            df = pd.read_csv(file_path)
            
            # Filter for baseline data
            baseline_data = df[(df['year'] == baseline_year) & (df['scenario'] == baseline_scenario)]
            baseline_total = baseline_data[metric].sum()
            
            # Dictionary to store results for this hazard
            hazard_results = {'Hazard': hazard, 'Baseline Total': baseline_total}
            totals['Baseline Total'] += baseline_total  # Accumulate baseline total
            
            for future_year in future_years:
                for future_scenario in future_scenarios:
                    # Filter for the future data
                    future_data = df[(df['year'] == future_year) & (df['scenario'] == future_scenario)]
                    future_total = future_data[metric].sum()
                    
                    # Calculate percentage change
                    if baseline_total > 0:
                        percentage_change = ((future_total - baseline_total) / baseline_total) * 100
                    else:
                        percentage_change = np.nan  # Handle zero baseline total
                    
                    # Add results to the hazard_results dictionary
                    abs_column_name = f"{future_scenario}_{future_year}_Value"
                    perc_column_name = f"{future_scenario}_{future_year}_Change (%)"
                    hazard_results[abs_column_name] = future_total
                    hazard_results[perc_column_name] = percentage_change
                    
                    # Accumulate totals for absolute values
                    totals[abs_column_name] += future_total

            # Append the results for this hazard
            results.append(hazard_results)

        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # Recalculate percentage changes for totals based on absolute values
    for future_year in future_years:
        for future_scenario in future_scenarios:
            abs_column_name = f"{future_scenario}_{future_year}_Value"
            perc_column_name = f"{future_scenario}_{future_year}_Change (%)"
            total_future_value = totals[abs_column_name]
            total_baseline_value = totals['Baseline Total']
            if total_baseline_value > 0:
                totals[perc_column_name] = ((total_future_value - total_baseline_value) / total_baseline_value) * 100
            else:
                totals[perc_column_name] = np.nan  # Handle zero baseline total

    # Append the totals row
    results.append(totals)

    # Create a summary DataFrame
    summary_df = pd.DataFrame(results)

    # Print the summary
    print(summary_df)

    return summary_df


In [4]:
input_files=[
    f'{results_path}tc_admin0_0.55_event-based.csv',
    f'{results_path}cf_admin0_0.55_check.csv',
    f'{results_path}FL_admin0.csv',
    f'{results_path}drought_admin0.csv'
]

future_years = [2050, 2100]
future_scenarios = ['optimistic', 'pessimistic']

summary = calculate_future_changes_multiple_scenarios(
    input_files=input_files,
    baseline_year=2020,
    baseline_scenario='current',
    future_years=future_years,
    future_scenarios=future_scenarios,
    metric='AAD'
)

                     Hazard  Baseline Total  optimistic_2050_Value  \
0                        tc    1.688406e+05           6.897092e+05   
1                        cf    1.025745e+07           1.124110e+07   
2                        FL    1.541652e+07           0.000000e+00   
3                   drought    4.324839e+06           1.324890e+07   
4  Total Across All Hazards    3.016765e+07           2.517972e+07   

   optimistic_2050_Change (%)  pessimistic_2050_Value  \
0                  308.497257            1.001257e+06   
1                    9.589657            1.152129e+07   
2                 -100.000000            0.000000e+00   
3                  206.344442            1.400793e+07   
4                  -16.534041            2.653048e+07   

   pessimistic_2050_Change (%)  optimistic_2100_Value  \
0                   493.019059           9.878875e+05   
1                    12.321172           1.426981e+07   
2                  -100.000000           2.296271e+07   
3       

In [5]:
summary.to_csv(f'{results_path}global_summaries.csv', index=False)

### Values referred to in the results section at various occasions

In [6]:
def create_per_capita_ranking_table(input_files, population_files, year, scenario, metric):
    """
    Create a table ranking countries independently for each hazard, using per capita values.
    
    Args:
    - input_files: List of CSV file paths, one for each hazard.
    - population_files: List of CSV file paths, one for each hazard's population data.
    - year: Time period to filter (e.g., 2020, 2050, 2100).
    - scenario: Scenario to filter (e.g., 'current', 'optimistic', 'pessimistic').
    - metric: Metric to rank by (e.g., 'AAD').
    - output_file: Path to save the resulting table as a CSV file.

    Returns:
    - DataFrame: Table with independent rankings for each hazard based on per capita values.
    """
    ranking_data = []
    hazard_names = []
    output_file=f'{results_path}per_capita_ranking_table_{year}_{scenario}_{metric}.csv'

    for csv_file, pop_file in zip(input_files, population_files):
        # Extract hazard name from file name
        hazard_name = csv_file.split("/")[-1].split(".")[0]  # Use file name as hazard name
        hazard_names.append(hazard_name)
        
        # Load population data specific to this hazard
        pop_df = pd.read_csv(pop_file)

        # Load and filter data
        df = pd.read_csv(csv_file)
        df = df.merge(pop_df, on='admin0', how='left')
        df['per_capita'] = df[metric] / df['valhum']
        df_filtered = df[(df['year'] == year) & (df['scenario'] == scenario)]
        
        # Rank countries by per capita value for the current hazard
        df_ranked = df_filtered[['admin0', 'per_capita']].sort_values(by='per_capita', ascending=False).reset_index(drop=True)
        df_ranked['Rank'] = df_ranked.index + 1
        
        # Append hazard-specific ranking data
        ranking_data.append(df_ranked[['Rank', 'admin0', 'per_capita']].rename(
            columns={'admin0': f'{hazard_name}_Country', 'per_capita': f'{hazard_name}_Per_Capita'}
        ))

    # Combine all rankings into a single table
    result = pd.concat(ranking_data, axis=1)
    
    # Keep only the first Rank column
    rank_column = result['Rank'].iloc[:, 0]
    result = result.drop(columns=[col for col in result.columns if col == 'Rank'])
    result.insert(0, 'Rank', rank_column)

    # Save the full ordered table to a CSV file
    result.to_csv(output_file, index=False)

    # Display only the top 10 rows
    top10 = result.head(10)
    print(top10)
    
    return result


In [7]:
def create_hazard_ranking_table(input_files, year, scenario, metric):
    """
    Create a table ranking countries independently for each hazard, with a single rank column.
    
    Args:
    - input_files: List of CSV file paths, one for each hazard.
    - year: Time period to filter (e.g., 2020, 2050, 2100).
    - scenario: Scenario to filter (e.g., 'current', 'optimistic', 'pessimistic').
    - metric: Metric to rank by (e.g., 'AAD').

    Returns:
    - DataFrame: Table with independent rankings for each hazard and a single Rank column.
    """
    ranking_data = []
    hazard_names = []
    output_file=f'{results_path}hazard_ranking_table_{year}_{scenario}_{metric}.csv'

    for csv_file in input_files:
        # Extract hazard name from file name
        hazard_name = csv_file.split("/")[-1].split(".")[0]  # Use file name as hazard name
        hazard_names.append(hazard_name)
        
        # Load and filter data
        df = pd.read_csv(csv_file)
        df_filtered = df[(df['year'] == year) & (df['scenario'] == scenario)]
        
        # Rank countries by displacement for the current hazard
        df_ranked = df_filtered[['admin0', metric]].sort_values(by=metric, ascending=False).reset_index(drop=True)
        df_ranked['Rank'] = df_ranked.index + 1
        
        # Append hazard-specific ranking data
        ranking_data.append(df_ranked[['Rank', 'admin0', metric]].rename(
            columns={'admin0': f'{hazard_name}_Country', metric: f'{hazard_name}_Displacement'}
        ))

    # Combine all rankings into a single table
    result = pd.concat(ranking_data, axis=1)
    
    # Keep only the first Rank column
    rank_column = result['Rank'].iloc[:, 0]
    result = result.drop(columns=[col for col in result.columns if col == 'Rank'])
    result.insert(0, 'Rank', rank_column)

    # Save the full ordered table to a CSV file
    result.to_csv(output_file, index=False)

    # Display only the top 10 rows
    top10 = result.head(10)
    print(top10)
    
    return result

In [8]:
def create_future_change_ranking_table(input_files, baseline_year, baseline_scenario, future_year, future_scenario, metric):
    """
    Create a table ranking countries independently for each hazard, based on future changes.
    
    Args:
    - input_files: List of CSV file paths, one for each hazard.
    - baseline_year: Year for the baseline scenario (e.g., 2020).
    - baseline_scenario: Scenario for the baseline (e.g., 'current').
    - future_year: Year for the future scenario (e.g., 2050).
    - future_scenario: Scenario for the future (e.g., 'optimistic').
    - metric: Metric to calculate relative change (e.g., 'AAD').
    - output_file: Path to save the resulting table as a CSV file.

    Returns:
    - DataFrame: Table with independent rankings for each hazard based on future changes.
    """
    ranking_data = []
    hazard_names = []
    output_file=f'{results_path}future_change_ranking_table_{future_year}_{future_scenario}_{metric}.csv'
    
    for csv_file in input_files:
        # Extract hazard name from file name
        hazard_name = csv_file.split("/")[-1].split(".")[0]  # Use file name as hazard name
        hazard_names.append(hazard_name)
        
        # Load and filter data
        df = pd.read_csv(csv_file)
        baseline_data = df[(df['year'] == baseline_year) & (df['scenario'] == baseline_scenario)]
        future_data = df[(df['year'] == future_year) & (df['scenario'] == future_scenario)]
        
        # Merge baseline and future data
        merged = pd.merge(
            baseline_data[['admin0', metric]],
            future_data[['admin0', metric]],
            on='admin0',
            suffixes=('_baseline', '_future')
        )
        
        # Calculate relative change
        merged['relative_change'] = (merged[f'{metric}_future'] - merged[f'{metric}_baseline']) / merged[f'{metric}_baseline'] * 100
        merged['relative_change'] = merged['relative_change'].replace([float('inf'), -float('inf')], np.nan)

        # Rank countries by relative change
        df_ranked = merged[['admin0', 'relative_change']].sort_values(by='relative_change', ascending=False).reset_index(drop=True)
        df_ranked['Rank'] = df_ranked.index + 1
        
        # Append hazard-specific ranking data
        ranking_data.append(df_ranked[['Rank', 'admin0', 'relative_change']].rename(
            columns={'admin0': f'{hazard_name}_Country', 'relative_change': f'{hazard_name}_Change (%)'}
        ))

    # Combine all rankings into a single table
    result = pd.concat(ranking_data, axis=1)
    
    # Keep only the first Rank column
    rank_column = result['Rank'].iloc[:, 0]
    result = result.drop(columns=[col for col in result.columns if col == 'Rank'])
    result.insert(0, 'Rank', rank_column)

    # Save the full ordered table to a CSV file
    result.to_csv(output_file, index=False)

    # Display only the top 10 rows
    top10 = result.head(10)
    print(top10)
    
    return result


In [9]:
ranking_table = create_hazard_ranking_table(
    input_files=[
        f'{results_path}tc_admin0_0.55_event-based.csv',
        f'{results_path}cf_admin0_0.55_check.csv',
        f'{results_path}FL_admin0.csv',
        f'{results_path}drought_admin0.csv'
    ],
    year=2020,
    scenario='current',
    metric='AAD'
)

   Rank tc_admin0_0_Country  tc_admin0_0_Displacement cf_admin0_0_Country  \
0     1                 PHL             104316.771399                 NLD   
1     2                 USA              28671.968123                 BGD   
2     3                 CHN               7300.799813                 USA   
3     4                 VNM               5535.861263                 CHN   
4     5                 THA               5079.057371                 EGY   
5     6                 IND               2995.845246                 JPN   
6     7                 TZA               2637.480909                 IND   
7     8                 HTI               2579.378834                 PHL   
8     9                 MOZ               1323.504145                 ITA   
9    10                 CUB                894.972221                 CAN   

   cf_admin0_0_Displacement FL_admin0_Country  FL_admin0_Displacement  \
0              7.326491e+06               VNM            2.643728e+06   
1     

In [10]:
per_capita_table = create_per_capita_ranking_table(
    input_files=[
        f'{results_path}tc_admin0_0.55_event-based.csv',
        f'{results_path}cf_admin0_0.55_check.csv',
        f'{results_path}FL_admin0.csv',
        f'{results_path}drought_admin0.csv'
    ],
    population_files = [
        f'{results_path}pop_count_admin0.csv',
        f'{results_path}pop_count_admin0.csv',
        f'{results_path}pop_count_admin0.csv',
        f'{results_path}pop_count_admin0_drought.csv'
    ],
    year=2020,
    scenario='current',
    metric='PMD_100'
)

   Rank tc_admin0_0_Country  tc_admin0_0_Per_Capita cf_admin0_0_Country  \
0     1                 PHL                0.014890                 NLD   
1     2                 USA                0.000885                 BGD   
2     3                 VNM                0.000652                 DEU   
3     4                 SLB                0.000533                 PHL   
4     5                 THA                0.000272                 USA   
5     6                 TZA                0.000237                 JPN   
6     7                 MOZ                0.000199                 EGY   
7     8                 MDG                0.000167                 LVA   
8     9                 MEX                0.000033                 BEL   
9    10                 AUS                0.000028                 CAN   

   cf_admin0_0_Per_Capita FL_admin0_Country  FL_admin0_Per_Capita  \
0                0.222842               KHM              0.042998   
1                0.039750           

In [11]:
future_change_table = create_future_change_ranking_table(
    input_files=[
        f'{results_path}tc_admin0_0.55_event-based.csv',
        f'{results_path}cf_admin0_0.55_check.csv',
        f'{results_path}FL_admin0.csv',
        f'{results_path}drought_admin0.csv'
    ],
    baseline_year=2020,
    baseline_scenario='current',
    future_year=2100,
    future_scenario='optimistic',
    metric='AAD'
)


   Rank tc_admin0_0_Country  tc_admin0_0_Change (%) cf_admin0_0_Country  \
0     1                 MDV            9.153166e+07                 SUR   
1     2                 WSM            2.140673e+05                 MRT   
2     3                 BHS            4.298425e+04                 THA   
3     4                 CAN            1.861781e+04                 SLE   
4     5                 TWN            1.258860e+04                 GTM   
5     6                 MYS            4.276025e+03                 JAM   
6     7                 MAC            1.875961e+03                 KHM   
7     8                 KHM            1.626701e+03                 MAR   
8     9                 LKA            1.463067e+03                 ZAF   
9    10                 TLS            1.187245e+03                 PER   

   cf_admin0_0_Change (%) FL_admin0_Country  FL_admin0_Change (%)  \
0           746812.904652               ITA            512.742293   
1           369819.989105           